In [ ]:
#|default_exp data.image

# Time Series Imaging> Convert time series to images for CNN-based analysis

This module provides functions to convert time series into 2D image representations:- **GASF** (Gramian Angular Summation Field)- **GADF** (Gramian Angular Difference Field)- **Recurrence Plots**- **MTF** (Markov Transition Field)

In [ ]:
import tsai_rsimport numpy as np

## Gramian Angular FieldsGramian Angular Fields encode time series as images using trigonometric functions.

In [ ]:
# Load and preprocess sample dataX_train, y_train, X_test, y_test = tsai_rs.get_UCR_data('ECG200')X_train_std = tsai_rs.ts_standardize(X_train)print(f"Input shape: {X_train_std.shape}")

In [ ]:
# Compute Gramian Angular Summation Field (GASF)gasf = tsai_rs.compute_gasf(X_train_std)print(f"GASF shape: {gasf.shape}")print(f"GASF value range: [{gasf.min():.4f}, {gasf.max():.4f}]")

In [ ]:
# Compute Gramian Angular Difference Field (GADF)gadf = tsai_rs.compute_gadf(X_train_std)print(f"GADF shape: {gadf.shape}")print(f"GADF value range: [{gadf.min():.4f}, {gadf.max():.4f}]")

## Recurrence PlotsRecurrence plots visualize the recurrence of states in phase space.

In [ ]:
# Compute Recurrence Plotrp = tsai_rs.compute_recurrence_plot(X_train_std)print(f"Recurrence Plot shape: {rp.shape}")print(f"RP value range: [{rp.min():.4f}, {rp.max():.4f}]")

## Understanding the Transforms### GASF (Gramian Angular Summation Field)The GASF represents the temporal correlation between different time points using the sum of angular values.### GADF (Gramian Angular Difference Field)The GADF represents the temporal correlation using the difference of angular values.### Recurrence PlotA recurrence plot shows the times at which the phase space trajectory visits roughly the same area in the phase space.

In [ ]:
# Example: Visualize dimensionssample_idx = 0channel_idx = 0x = X_train_std[sample_idx, channel_idx, :]print(f"Original time series length: {len(x)}")print(f"GASF image size: {gasf[sample_idx, channel_idx].shape}")print(f"GADF image size: {gadf[sample_idx, channel_idx].shape}")print(f"RP image size: {rp[sample_idx, channel_idx].shape}")

## Using Images with CNNsThese image representations can be used as input to standard image classification models.

In [ ]:
def ts_to_images(X, method='gasf'):    """Convert time series to images.        Args:        X: Time series data [n_samples, n_channels, seq_len]        method: 'gasf', 'gadf', 'rp', or 'all'        Returns:        Image representation(s)    """    if method == 'gasf':        return tsai_rs.compute_gasf(X)    elif method == 'gadf':        return tsai_rs.compute_gadf(X)    elif method == 'rp':        return tsai_rs.compute_recurrence_plot(X)    elif method == 'all':        gasf = tsai_rs.compute_gasf(X)        gadf = tsai_rs.compute_gadf(X)        rp = tsai_rs.compute_recurrence_plot(X)        # Stack along channel dimension        return np.concatenate([gasf, gadf, rp], axis=1)    else:        raise ValueError(f"Unknown method: {method}")

In [ ]:
# Convert to multi-channel imagesX_images = ts_to_images(X_train_std[:10], method='all')print(f"Multi-channel image shape: {X_images.shape}")print(f"(n_samples, n_channels * 3, height, width)")

## Memory-Efficient ProcessingFor large datasets, process in batches to manage memory usage.

In [ ]:
def batch_ts_to_images(X, method='gasf', batch_size=32):    """Convert time series to images in batches."""    n = len(X)    results = []        for start in range(0, n, batch_size):        end = min(start + batch_size, n)        batch = X[start:end]        images = ts_to_images(batch, method=method)        results.append(images)        return np.concatenate(results, axis=0)# Example usageX_gasf = batch_ts_to_images(X_train_std, method='gasf', batch_size=50)print(f"Processed GASF shape: {X_gasf.shape}")